# Spring 2022
# CPSC 585 Project 4
## Raymond Carpio
## Yu Pan
## Sijie Shang
## John Tu

### Based on the example from Tensorflow's text generation with RNN:
### https://www.tensorflow.org/text/tutorials/text_generation

### Text used for the RNN model: The entire script from Lord of the Rings: Fellowship of the Ring

# Changes made to the code:
## If there are modifications made for this code, then it would be the length of the example sequence for input text processing and the number of epochs to run for training. Also, the changes made so far are the example text vector and the temperature parameter for the one-step RNN model class.

In [ ]:
# Import the following modules over
import numpy as np
import tensorflow as tf
import os
import time

# Try to open the text file if it exists.
input_file = "rings.txt"
corpus_raw = open(input_file, "r", encoding="utf-8").read()

# Print out the sample text file.
#print(corpus_raw)

In [ ]:
# Obtain the list of characters included in the raw text.
# Be sure to use set() to filter out duplicates.
characters = sorted(list(set(corpus_raw)))

# Print out the list of characters found in the raw text.
print(characters)

# Also obtain the total length of the text and the characters.
print("Total length of text: ", len(corpus_raw))
print("Total number of characters found: ", len(characters))

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Ó', 'á', 'â', 'ä', 'é', 'ë', 'í', 'ó', 'ú', 'û', '–']
Total length of text:  1021058
Total number of characters found:  90


In [ ]:
# Print out the first 500 characters of the text.
print(corpus_raw[:500])

Three Rings for the Elven-kings under the sky,
               Seven for the Dwarf-lords in their halls of stone,
            Nine for Mortal Men doomed to die,
              One for the Dark Lord on his dark throne
           In the Land of Mordor where the Shadows lie.
               One Ring to rule them all, One Ring to find them,
               One Ring to bring them all and in the darkness bind them
           In the Land of Mordor where the Shadows lie.
           
FOREWORD

This tale grew


In [ ]:
# Vectorize the text by converting the string into a numerical form.
sample_text = ['abcdefghij', 'wxyz']
num_chars = tf.strings.unicode_split(sample_text, input_encoding="UTF-8")
num_chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j'],
 [b'w', b'x', b'y', b'z']]>

In [ ]:
id_from_characters = tf.keras.layers.StringLookup(vocabulary=list(characters))
ids = id_from_characters(num_chars)
ids

<tf.RaggedTensor [[54, 55, 56, 57, 58, 59, 60, 61, 62, 63], [76, 77, 78, 79]]>

In [ ]:
characters_from_id = tf.keras.layers.StringLookup(vocabulary=id_from_characters.get_vocabulary(), invert=True)
characters = characters_from_id(ids)
characters

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j'],
 [b'w', b'x', b'y', b'z']]>

In [ ]:
# Convert the corresponding string ids back into text form.
def text_from_id(ids):
    return tf.strings.reduce_join(characters_from_id(ids), axis=-1)

In [ ]:
all_text_id = id_from_characters(tf.strings.unicode_split(corpus_raw, 'UTF-8'))
all_text_id

<tf.Tensor: shape=(1021058,), dtype=int64, numpy=array([45, 61, 71, ..., 32, 10,  1])>

In [ ]:
# Try to print out the first 25 characters from raw text via tensor slices.
id_dataset = tf.data.Dataset.from_tensor_slices(all_text_id)
for curr_id in id_dataset.take(25):
    print(characters_from_id(curr_id).numpy().decode('utf-8'))

T
h
r
e
e
 
R
i
n
g
s
 
f
o
r
 
t
h
e
 
E
l
v
e
n


In [ ]:
# Now build a sequence of the first 200 characters.
sequence_len = 200
each_example = len(corpus_raw)
sequences = id_dataset.batch(sequence_len+1, drop_remainder=True)

for curr_seq in sequences.take(1):
    print(characters_from_id(curr_seq))

tf.Tensor(
[b'T' b'h' b'r' b'e' b'e' b' ' b'R' b'i' b'n' b'g' b's' b' ' b'f' b'o'
 b'r' b' ' b't' b'h' b'e' b' ' b'E' b'l' b'v' b'e' b'n' b'-' b'k' b'i'
 b'n' b'g' b's' b' ' b'u' b'n' b'd' b'e' b'r' b' ' b't' b'h' b'e' b' '
 b's' b'k' b'y' b',' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b'S' b'e' b'v' b'e' b'n' b' ' b'f' b'o'
 b'r' b' ' b't' b'h' b'e' b' ' b'D' b'w' b'a' b'r' b'f' b'-' b'l' b'o'
 b'r' b'd' b's' b' ' b'i' b'n' b' ' b't' b'h' b'e' b'i' b'r' b' ' b'h'
 b'a' b'l' b'l' b's' b' ' b'o' b'f' b' ' b's' b't' b'o' b'n' b'e' b','
 b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'N'
 b'i' b'n' b'e' b' ' b'f' b'o' b'r' b' ' b'M' b'o' b'r' b't' b'a' b'l'
 b' ' b'M' b'e' b'n' b' ' b'd' b'o' b'o' b'm' b'e' b'd' b' ' b't' b'o'
 b' ' b'd' b'i' b'e' b',' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b'O' b'n' b'e' b' ' b'f' b'o' b'r' b' '
 b't' b'h' b'e' b' ' b'D' b'a' b'r' b'k' b' ' b'L' b'o' b'r' b'

In [ ]:
for curr_seq in sequences.take(5):
    print(text_from_id(curr_seq).numpy())

b'Three Rings for the Elven-kings under the sky,\n               Seven for the Dwarf-lords in their halls of stone,\n            Nine for Mortal Men doomed to die,\n              One for the Dark Lord on hi'
b's dark throne\n           In the Land of Mordor where the Shadows lie.\n               One Ring to rule them all, One Ring to find them,\n               One Ring to bring them all and in the darkness bind'
b' them\n           In the Land of Mordor where the Shadows lie.\n           \nFOREWORD\n\nThis tale grew in the telling, until it became a history of the Great War of the Ring and included many glimpses of t'
b'he yet more ancient history that preceded it. It was begun soon after _The Hobbit_ was written and before its publication in 1937; but I did not go on with this sequel, for I wished first to complete a'
b'nd set in order the mythology and legends of the Elder Days, which had then been taking shape for some years. I desired to do this for my own satisfaction, and I had l

In [ ]:
def split_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
text_dataset = sequences.map(split_target)
for input_text, target_text in text_dataset.take(1):
    print("Input: ", text_from_id(input_text).numpy())
    print("Target: ", text_from_id(target_text).numpy())

Input:  b'Three Rings for the Elven-kings under the sky,\n               Seven for the Dwarf-lords in their halls of stone,\n            Nine for Mortal Men doomed to die,\n              One for the Dark Lord on h'
Target:  b'hree Rings for the Elven-kings under the sky,\n               Seven for the Dwarf-lords in their halls of stone,\n            Nine for Mortal Men doomed to die,\n              One for the Dark Lord on hi'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    text_dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 200), dtype=tf.int64, name=None), TensorSpec(shape=(64, 200), dtype=tf.int64, name=None))>

In [ ]:
vocab = sorted(set(corpus_raw))
vocab_size = len(vocab)
embedding_dimension=256
rnn_units=1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(id_from_characters.get_vocabulary()),
    embedding_dim=embedding_dimension,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 200, 91) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  23296     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  93275     
                                                                 
Total params: 4,054,875
Trainable params: 4,054,875
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([61, 31, 72, 70,  3, 63, 46, 24, 70, 43, 13, 56, 79, 62, 72, 81, 51,
       86, 44, 30, 67, 89,  9,  2, 65,  4, 58, 41, 60, 78, 19, 59, 31, 50,
       60, 50, 16, 63, 22, 44, 30,  1, 61, 27,  8,  3, 87, 82,  5, 88, 37,
       27, 22, 87, 32, 65, 34, 56, 47, 44, 60,  6,  8, 69, 24, 34, 30, 86,
        1, 59, 36, 37,  3,  1, 33,  5, 87, 32,  9,  4, 27, 36, 36, 68,  5,
       18, 64, 40, 55, 56, 54, 13, 76, 25, 78, 12, 41, 31,  7, 88, 58, 55,
       55, 36, 80, 59, 44, 90, 46, 31, 67, 57, 89, 13, 76, 51, 42,  7, 25,
       16, 31, 54, 34, 50, 64, 59, 56, 42, 84, 17, 18, 53, 32, 21, 49, 62,
       11, 46, 62, 65, 58, 30, 20,  8, 84, 49, 20, 58, 46, 39,  7, 46, 88,
       10, 64, 41, 34, 45, 76, 32, 59, 75, 16, 78, 61, 43, 30, 89, 44, 64,
       39, 35, 32, 15, 89, 23, 21, 84, 32, 15, 53, 31, 16, 33, 86, 41, 13,
        4,  6, 62, 32, 17, 24, 38, 37, 83, 55, 39, 76, 22])

In [ ]:
print("Input:\n", text_from_id(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_id(sampled_indices).numpy())

Input:
 b'  _Chapter 2_\n            The Shadow of the Past\n\n     The talk did not die down in nine or even ninety-nine days. The second disappearance of Mr. Bilbo Baggins was discussed in Hobbiton, and indeed a'

Next Char Predictions:
 b'hFsq!jU=qR1czis\xc3\xa1Z\xc3\xadSEn\xc3\xbb- l"ePgy7fFYgY4j:SE\nhB,!\xc3\xb3\xc3\xa2\'\xc3\xbaLB:\xc3\xb3GlIcVSg(,p=IE\xc3\xad\nfKL!\nH\'\xc3\xb3G-"BKKo\'6kObca1w?y0PF)\xc3\xbaebbK\xc3\x93fS\xe2\x80\x93UFnd\xc3\xbb1wZQ)?4FaIYkfcQ\xc3\xa956`G9Xi/UileE8,\xc3\xa9X8eUN)U\xc3\xba.kPITwGfv4yhRE\xc3\xbbSkNJG3\xc3\xbb;9\xc3\xa9G3`F4H\xc3\xadP1"(iG5=ML\xc3\xa4bNw:'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()

Prediction shape:  (64, 200, 91)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.5113063, shape=(), dtype=float32)


91.04067

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
79/79 [==============================] - 10s 92ms/step - loss: 3.0312
Epoch 2/100
79/79 [==============================] - 8s 91ms/step - loss: 2.2180
Epoch 3/100
79/79 [==============================] - 8s 91ms/step - loss: 1.9455
Epoch 4/100
79/79 [==============================] - 8s 92ms/step - loss: 1.7320
Epoch 5/100
79/79 [==============================] - 8s 93ms/step - loss: 1.5665
Epoch 6/100
79/79 [==============================] - 9s 94ms/step - loss: 1.4466
Epoch 7/100
79/79 [==============================] - 9s 94ms/step - loss: 1.3598
Epoch 8/100
79/79 [==============================] - 9s 95ms/step - loss: 1.2953
Epoch 9/100
79/79 [==============================] - 9s 95ms/step - loss: 1.2432
Epoch 10/100
79/79 [==============================] - 9s 95ms/step - loss: 1.2019
Epoch 11/100
79/79 [==============================] - 9s 96ms/step - loss: 1.1649
Epoch 12/100
79/79 [==============================] - 9s 96ms/step - loss: 1.1322
Epoch 13/100
79/79 [====

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, characters_from_id, id_from_characters, 1.0)
one_step_model1 = OneStep(model, characters_from_id, id_from_characters, 0.9)
one_step_model2 = OneStep(model, characters_from_id, id_from_characters, 0.8)
one_step_model3 = OneStep(model, characters_from_id, id_from_characters, 0.7)
one_step_model4 = OneStep(model, characters_from_id, id_from_characters, 0.6)
one_step_model5 = OneStep(model, characters_from_id, id_from_characters, 0.5)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There was a sound of munning eastwards behind, or halfling now to make loss lands for them:
     There were no power over the mountains of Saruman dimly into the touch for many times; but the wind in the familiest standing stone like step forward. In the dear of the reperth ninety ninet, they ming, free for the outching through the lowly into a deeply drum-beat:
     Then he would not slipping flask and asked to wait. 'That is the way for us,' said Gimli.
     `I have not the sense of proof,' said Frodo, 'for wh can have been destroyed. The Ringwraiths are destroyed before I desire to keep about all this, and it must go down in the world outside. Gandalf walked in flacks, and welcomed to die, and I understand I call a pony up; but befulted was the chieftap and a cold likely tonsumerth. His farm was hew walking, eastward, but they were searching to the best head and thrust without a struggle. It was difficult to be waiting back to him, and he listened without a string, no less channel t

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model1.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There was a sound of wonder and all the others and put in keeping surprisets, for they felt much between Stock and Legolas.
     They strode forward from the top pass over the rings above and on either side, and they could not see them away. The valley seemed vanished with a faint step that was now and again; but they were more perilous treasure. And if that was not enough north of Rivendell, and returned to their pale paths broken.
     This would turn to start upon the evil of the Downs and flowers, as they breathed. Turning back in the Moon is ran away to the west. In that way they would steaming hostilarcusts and the Lady stopped and waved his cushing from the East. 'Black Riders!' akknow as I caught ' he cried. 'Did I not say. They are only waiting.'
     In the morning they went on, too, off the ring had already beaut it and proudly in the darkness.
     Frodo laughed green and brows to life.
     Twilight filled with a traveller on his ribboush, and leaned against white wolves i

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model2.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There was a sound of honest lands, or inside. It sought cat still passed to and fro obsiness, Kingdom and Pippin rusked by another thing on for a nice later to Gandalf. Thurs a thing or two others, many times spoken to the floor, and one on either side.
     Merry appair sleep leaves, as if strange their bedrooms.
     When they caught his words again they found thanks and fell fast asleep at the foot of the stems. It leaned over him. He thought there were twory will get to keep a bit about a _preasunery-giest of wolves, or disturbed me. I never be threw himself from the opposite without there seemed to an incompinion.
     `For I must go, between news of the tales, suncernts away for us in this land, now know? " he said at last, 'I was I think, and I shall not be the master: but I have seldom reached here and all that went with you.
           Hey now shone and former low; but that flowed down red behind the mists red Borgir, Ere long, farm, crossing as easily as possible and secret g

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model3.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There was something in the look of the fire, they were sea, and below himself with his finger he looked round away, and the shadow of the enemy city of thus to strike the boat outside, and passing through a distand plain below it. A brief ran ahtriling they passed, and they seemed not to travelven swift, and the leaves on some way white girtt fell. But he would not until they found a spring of elven-white, and down news looking scraplich upon the long shadows of stones were laid on much arm us: a strong splun. A suddenly realm in the South, and as the ring is dangerous, far down roots (in came the time was passing through a hollow of the landlord. Pippin and Sam came up with him. It was kept at Great Smials, but it was not by now, and when they had entered the ring had given them for a while feet a little, and they saw the river below him, and they returned to their people. Long ago they found a strick broad sleep, two grey silver, under the night in deep wold, but they came to a narro

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model4.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There was a sound of honeshing in the floory, and once they stood, studied it off along the road. There they stood aloot as they bound to the top of a steep bank in a wearing sound, filled with them that shich with water climbed up the stone reluctant that they saw the travellers approached the old in Boromir; and I was there so long ago that the hills have I returned. 'We have any boat before you said, changing still, and many even from the Bree-land and the mountains!' he said suddenly and saw that Land the valley sheet high and looked at him. 'It was this Mr. Frodo and I real your footstench. One by one who  is near the figure to kind it is,' said Legolas.
     `A dwarf! ' said Sam. `I thought it was only for a long time to me than that, I can.' Hope Fatty Bolger had joined the know swift to late and holding and a shout of his lips had in rear,
           And in the gland singled spell.
     'What's that? ' cried Gandalf, spring and went and struggle up a stirl. 'When we go to the w

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model5.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There was a sound of much the same things and tosssars that could not be found; and in the same combort was made lay.
     'And what has the strength, and you cannot enough, Mr. Underhill?' he asked. 'It wouldn't keep a horrible new, but to give you away; but I guess that in that cries would not be dark before now. As the road that I seek for him, even when he is strong or badgest two barges and deads to presive needs that I cannot linger than travellers. Grove from the Shire! '
     'Ride all things and places now the passage from the Seven and the Old, `not at any rate here. I may have started your touched without rest. And one thing were to do without relating about his father's tired. Enormous in the northward trees. They won't get it. I won't give my precious away, I tell you,' said Celeborn. `So you go on by rider by the news after those two youn holes, and I could not "know,' answered Gandalf, 'but before that he had wandered far, free to answer at once. And yet well enough do n